<a href="https://colab.research.google.com/github/chaerui7967/Netflix_Analysis_individual/blob/master/%EC%A0%90%EC%88%98_%EC%98%88%EC%B8%A1%EB%AA%A8%EB%8D%B8_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%ACipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


import matplotlib.pyplot as plt
import missingno # 널값 바차트로 시각화

In [ ]:
netflix = pd.read_csv('./use_df.csv')

In [ ]:
netflix.head(1)

In [ ]:
netflix.columns

In [ ]:
df_net = netflix.drop(columns = ['Unnamed: 0'],axis =1)

In [ ]:
df_net.isnull().sum()

In [ ]:
color= ['dimgrey','dimgrey','blue','blue','blue','dimgrey','blue','blue','blue','blue','blue',
        'dimgrey','blue','blue','blue','blue','blue','blue','blue','blue','blue','blue','dimgrey',
        'blue','blue','blue']
missingno.bar(df_net,fontsize=10,color=color,figsize=(10,5))
plt.title('COLUMN MISSING VALUES',fontsize=20)

movie viewpoint 예측에 사용될 변수들을 일단은 국가와 장르, 태그, 언어, 출시 일, 런타임을 가지고 먼저 예측초기모델을 만륻어보자

In [ ]:
movie_df = df_net[df_net['Series or Movie']=='Movie']

In [ ]:
movie_df = movie_df[['COUNTRY', 'Release Date', 'GENRE', 'Tags', 'Languages', 'Runtime']]

In [ ]:
pd_df = pd.get_dummies(netflix['subject'])
df_result = pd.concat([netflix, pd_df], axis=1)